In [24]:
import ast
from collections import Counter
import math
import zlib
import pandas as pd
import numpy as np 
import mido

In [3]:
output_directory = "music_metrics.csv"

In [4]:
def extract_features(notes_dict):
    pitches = [note[0] for note in notes_dict.values()]
    durations = [note[1] for note in notes_dict.values()]
    velocities = [note[2] for note in notes_dict.values()]
    return pitches, durations, velocities
def calculate_entropy(sequence):
    counts = Counter(sequence)
    probabilities = [count / len(sequence) for count in counts.values()]
    return -sum(p * math.log2(p) for p in probabilities if p > 0)
def calculate_compression_ratio(sequence):
    # Convert the sequence to a string for compression
    sequence_str = ",".join(map(str, sequence))
    compressed = zlib.compress(sequence_str.encode('utf-8'))
    return len(compressed) / len(sequence_str.encode('utf-8'))

def calculate_metrix(df):
    complexity_scores = []
    for _, row in df.iterrows():
        notes_dict = ast.literal_eval(row["Notes"])  # Convert string back to dictionary
        pitches, durations, velocities = extract_features(notes_dict)

        metrics = {
        "pitch_entropy": calculate_entropy(pitches),
        "duration_entropy": calculate_entropy(durations),
        "velocity_entropy": calculate_entropy(velocities),
        "pitch_compression": calculate_compression_ratio(pitches),
        "duration_compression": calculate_compression_ratio(durations),
        "velocity_compression": calculate_compression_ratio(velocities),
        }

        complexity_scores.append(metrics)

    # Add the complexity column to the dataframe
    return complexity_scores

In [7]:
df = pd.read_csv(output_directory)
df["Metrix"] = calculate_metrix(df)
print(df["Metrix"])

0       {'pitch_entropy': 4.039494663005557, 'duration...
1       {'pitch_entropy': 3.3698373541095803, 'duratio...
2       {'pitch_entropy': 3.6740738464145557, 'duratio...
3       {'pitch_entropy': 4.404917585176534, 'duration...
4       {'pitch_entropy': 4.484162704131576, 'duration...
                              ...                        
2031    {'pitch_entropy': 4.346988230461985, 'duration...
2032    {'pitch_entropy': 4.517722105737881, 'duration...
2033    {'pitch_entropy': 4.0377172942187, 'duration_e...
2034    {'pitch_entropy': 3.960525399817489, 'duration...
2035    {'pitch_entropy': 4.226813284500464, 'duration...
Name: Metrix, Length: 2036, dtype: object


In [6]:
df.shape

(2036, 7)

In [10]:
def extract_notes_from_df(df):
    # Crée trois nouvelles colonnes pour les pitches, durations et velocities
    df["pitches"] = df["Notes"].apply(lambda x: [note[0] for note in ast.literal_eval(x).values()])
    df["durations"] = df["Notes"].apply(lambda x: [note[1] for note in ast.literal_eval(x).values()])
    df["velocities"] = df["Notes"].apply(lambda x: [note[2] for note in ast.literal_eval(x).values()])
    return df


df = extract_notes_from_df(df)

In [16]:
#l = [df["pitches"][0],df["durations"][0],df["velocities"][0]]


print(df["pitches"][0][0]) #df["pitches"] est en 2D

55


Il n'y a que des partitions de piano dans le dataset, une seule piste à chaque morceau/ligne (donc ce qu'on voulait nice)

In [29]:
#Je veux des listes des triplets 


def extract_triplets(df):
    """
    Convert the 'Notes' column in the DataFrame into a list of triplets (pitches, durations, velocities).
    
    Args:
        df (pd.DataFrame): DataFrame containing a 'Notes' column with dictionary-like strings.
        
    Returns:
        pd.DataFrame: Updated DataFrame with a new column 'triplets' containing the list of tuples.
    """
    df["triplets"] = df["Notes"].apply(lambda x: [tuple(note) for note in ast.literal_eval(x).values()])
    return df

In [30]:
df = extract_triplets(df)

Approche de la complexité de Kolmogorov
En gros une note c'est un triplet (p,d,v) (pitch, duratio,velocity)
Donc deux notes identiques c'est deux notes qui ont les mêmes p,d,v
Si on a deux mêmes p,d,v alors on a juste à écrire la note une fois, et à mettre ses 2 emplacements. C'est moins long que d'encoder deux fois l'emplacement et le p,d,v. Donc premier facteur de réduction de complexité


Savoir qu'il y a plusieurs notes de même p ça sert pas forcément à grand chose car il faut quand même noter leur emplacement
Mais quand plusieurs notes de même p ou d ou v se suivent la c'est intéressant et ça fait diminuer la complexité
Donc quand on a des notes qui se suivent avec mêmes p ou d ou v, complexité diminue

pareil si c'est pour des couples de var : si deux notes se suivent avec même p,v (mais d différent) alors ça fait quand même bien diminuer la complexité 
Algo : 

1) on imagine que notre code code toutes les notes uniques et sans aucunes répetitions de p,d,v à droite ou à gauche, avec leurs emplacements

2) puis il prend celles qui se répètentsur une ou 2 vars, et les codes ensemble

3) puis il prend les notes identiques et code juste leur emplacement

Si une note est à la fois exactement égale à une autre, et apparait dans une répétition de p, alors comment on l'encode ? Ca c'est une bonne question (elle appartient aux notes de catégorie 2 et 3 à la fois)
-> Pour l'instant on considère que la méthode 3 est toujours plus efficace, donc on classe ce genre de note dans la classe 3

--> En vérité ça va dépendre de la longueur de la répétition d'une var. Si ça fait beaucoup diminuer la complexité des notes voisines alors il faudrait prendre encodage 2. Mais y a aussi un problème avec cette espèce de "valeur" de complexité. On fait des +1 -1 genre ?

Pour l'instant oui 

Plus long encodage : chaque triplet et son emplacement 

Taille de l'encodage d'un triplet (𝑝,𝑑,𝑣) : constante 𝐶_note
Taille de l'encodage de la position : constant 𝐶_position
nombre de notes : n 
Taille maximale de l'encodage : n x (𝐶_note + 𝐶_position) 

In [ ]:
mean_length = np.mean([len(df["triplets"][i]) for i in range(len(df["triplets"]))])
print(mean_length)

max_length = np.max([len(df["triplets"][i]) for i in range(len(df["triplets"]))])
print(max_length)

taille encodage p : p varie de 0 à 127, il est donc écrit sur 7 bits

d en millisecondes, ne dépasse pas 10 000 ms :  14 bits

v varie de 0 à 127 : 7 bits

𝐶_note = 28 bits

la partition la plus longue comporte : 9824 notes
9824 se code sur 14 bits
donc la position se code sur 14 bits

𝐶_position = 14 bits

Taille maximale de l'encodage = n x 42 bits

il faut avoir une manière de préciser dans le code qu’on a une répétition et la taille de la répétition. Il y a 5 types de répétitions possibles : d, p , v ou de doublets (p,v) , (p,d) (d,v), ça se code sur 4 bits. Ca tombe bien rien ne fait 4 bits dans ce qui précéde, donc dans notre code cès qu'on voit 4 bits, ça veut dire qu'il y a répétition. Ca dit qu'est-ce qui est répété. Puis ensuite ça donne le nombre de répétitions. Pas sur que ce 'quand tu vois un mot de 4 bits c'est que c'est pour préciser une répétition donc tu sais ce que ça veut dire' soit très correct. On suppose que les mots sont séparés par des espaces. Est-ce qu'on peut faire cette hypothèse ? 

In [ ]:
def estim_complexite(partition) :
    """ partition : liste de tuples de 3 int. Chaque tuple représente une note
    """
    worst_complexity = 42*len(partition) #Comme expliqué plus haut 

    #détecter les répétitions de d, p , v ou de doublets (p,v) , (p,d) (d,v)
    